## Lake Superior Temperature Profiles - 2D & 3D

This notebook brings in temperature data points over a small range of depth & area of Lake Superior. The data is interpolated into various 3D Empirical Bayesian Kriging (EBK) surface & 2D Contour shapefiles using ArcGIS Pro tools. 

In [16]:
# Import modules 
import arcpy
import pandas as pd

# Set the workspace environment to local file geodatabase
arcpy.env.workspace = r'C:\Users\mmMary\Documents\Projects\Superior_Surface_Temp'

In [30]:
# Take a peek at csv table
GG = (r'C:\Users\mmMary\Documents\Projects\Superior_Surface_Temp\Data\Glider\GG_11jul_1st.csv')
df = pd.read_csv(GG)
df

,depth (m),precise_lat (degree_north),precise_lon (degree_east),temperature (Celsius),precise_time (UTC)
0,-0.35,46.802483,-91.733625,9.23,2011-07-01T01:23:35Z
1,-0.27,46.802483,-91.733625,9.23,2011-07-01T01:23:37Z
2,-0.20,46.802483,-91.733625,9.23,2011-07-01T01:23:39Z
3,-0.15,46.802483,-91.733625,9.23,2011-07-01T01:23:42Z
4,-0.04,46.802483,-91.733625,9.23,2011-07-01T01:23:44Z
...,...,...,...,...,...
5593,1.50,46.859743,-91.770745,8.73,2011-07-01T08:31:01Z
5594,1.67,46.859746,-91.770743,8.72,2011-07-01T08:31:02Z
5595,1.84,46.859751,-91.770739,8.70,2011-07-01T08:31:04Z
5596,2.01,46.859756,-91.770736,8.69,2011-07-01T08:31:07Z


### Convert table coordinates into map points & project into local coordinates

In [38]:
# Export csv table to points
arcpy.management.XYTableToPoint("GG_11jul_1st.csv", r"C:\Users\mmMary\Documents\Projects\Superior_Surface_Temp\Superior_Surface_Temp_Map\Superior_Surface_Temp_Map.gdb\GG1_11jul_1st", "precise_lon (degree_east)", "precise_lat (degree_north)", "depth (m)", "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],VERTCS['WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PARAMETER['Vertical_Shift',0.0],PARAMETER['Direction',1.0],UNIT['Meter',1.0]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision")

<Result 'C:\\Users\\mmMary\\Documents\\Projects\\Superior_Surface_Temp\\Superior_Surface_Temp_Map\\Superior_Surface_Temp_Map.gdb\\GG1_11jul_1st'>

In [40]:
# Project new map points into UTM NAD 83 Zone 15
arcpy.management.Project("GG1_11jul_1st", r"C:\Users\mmMary\Documents\Projects\Superior_Surface_Temp\Superior_Surface_Temp_Map\Superior_Surface_Temp_Map.gdb\GGlider_Pts", "PROJCS['NAD_1983_2011_UTM_Zone_15N',GEOGCS['GCS_NAD_1983_2011',DATUM['D_NAD_1983_2011',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Transverse_Mercator'],PARAMETER['False_Easting',500000.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',-93.0],PARAMETER['Scale_Factor',0.9996],PARAMETER['Latitude_Of_Origin',0.0],UNIT['Meter',1.0]]", "WGS_1984_(ITRF08)_To_NAD_1983_2011", "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],VERTCS['WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PARAMETER['Vertical_Shift',0.0],PARAMETER['Direction',1.0],UNIT['Meter',1.0]]", "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

<Result 'C:\\Users\\mmMary\\Documents\\Projects\\Superior_Surface_Temp\\Superior_Surface_Temp_Map\\Superior_Surface_Temp_Map.gdb\\GGlider_Pts'>

### Create 3D Empircal Baysien Kriging surface

This creates a predictive interpolation surface of temperatures through the water column. EBK has several options for surface type, transformation, semivariogram type and more. A few select interations are shown here but the best parameters will depend on the specific data. 

In [ ]:
# esri defaults of no transformation, Exponential semivariogram
arcpy.ga.EmpiricalBayesianKriging3D("GG_11jul_1st", "z", "temperature__Celsius_", "EBK", "METER", None, "EXPONENTIAL", "NONE", 100, 1, 100, "NONE", None, "NBRTYPE=Standard3D RADIUS=nan NBR_MAX=2 NBR_MIN=1 SECTOR_TYPE=TWELVE_SECTORS", None, "PREDICTION", 0.5, "EXCEED", None)

In [ ]:
# Exponential semivariogram with Log Empirical transformation
arcpy.ga.EmpiricalBayesianKriging3D("GG_11jul_1st", "z", "temperature__Celsius_", "EBK_Expon_Log", "METER", None, "EXPONENTIAL", "LOGEMPIRICAL", 100, 1, 100, "NONE", None, "NBRTYPE=Standard3D RADIUS=nan NBR_MAX=2 NBR_MIN=1 SECTOR_TYPE=TWELVE_SECTORS", None, "PREDICTION", 0.5, "EXCEED", None)

In [ ]:
# Power Semivariogram with no transformation
arcpy.ga.EmpiricalBayesianKriging3D("GG_11jul_1st", "z", "temperature__Celsius_", "EBK_Power", "METER", None, "POWER", "NONE", 100, 1, 100, "NONE", None, "NBRTYPE=Standard3D RADIUS=nan NBR_MAX=2 NBR_MIN=1 SECTOR_TYPE=TWELVE_SECTORS", None, "PREDICTION", 0.5, "EXCEED", None)

### Export EBK Results
This is a simple visualization of the variable at a specified depth as either contour
lines or filled polygons. This can also be done in Batch mode.

In [1]:
# Export shp of filled polygons as a static layer at a given depth
arcpy.ga.GALayerToContour("EBK_Power", "SAME_AS_LAYER", r"C:\Users\mmMary\Documents\Projects\ebk_power.shp", "DRAFT", "GEOMETRIC_INTERVAL", 10, [], None)

<Result 'C:\\Users\\mmMary\\Documents\\Projects\\ebk_power.shp'>

In [2]:
# Export static shp of contour lines indicating temp at 2.17 meters
arcpy.ga.GALayerToContour("EBK_Power", "CONTOUR", r"C:\Users\mmMary\Documents\Projects\contour2_17.shp", "DRAFT", "GEOMETRIC_INTERVAL", 10, [], "2.17 Meters")

<Result 'C:\\Users\\mmMary\\Documents\\Projects\\contour2_17.shp'>